#### 1. Import pandas library

In [84]:
import pandas as pd

#### 2. Import users table:

In [85]:
users_df=pd.read_csv('users_table.csv')

#### 3. Rename Id column to userId

In [86]:
users_df.rename(columns={'Id':'userId'}, inplace=True)

#### 4. Import posts table:

In [87]:
posts_df=pd.read_csv('posts_table.csv')

#### 5. Rename Id column to postId and OwnerUserId to userId

In [88]:
posts_df.rename(columns={'Id':'postId', "OwnerUserId" : "userId"}, inplace=True)

In [89]:
users_df.columns

Index(['userId', 'Reputation', 'CreationDate', 'DisplayName', 'LastAccessDate',
       'WebsiteUrl', 'Location', 'AboutMe', 'Views', 'UpVotes', 'DownVotes',
       'AccountId', 'Age', 'ProfileImageUrl'],
      dtype='object')

#### 6. Define new dataframes for users and posts with the following selected columns:
    **users columns**: userId, Reputation,Views,UpVotes,DownVotes
    **posts columns**: postId, Score,userId,ViewCount,CommentCount

In [90]:
users=users_df[["userId", "Reputation","Views","UpVotes","DownVotes"]]
posts=posts_df[["postId", "Score","userId","ViewCount","CommentCount"]]

#### 7. Merge both dataframes, users and posts. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [91]:
users_posts=users.merge(posts, on = "userId")

#### 8. How many missing values do you have in your merged dataframe? On which columns?

In [93]:
users_posts.isnull().sum()

userId              0
Reputation          0
Views               0
UpVotes             0
DownVotes           0
postId              0
Score               0
ViewCount       23572
CommentCount        0
dtype: int64

#### 9. You will need to make something with missing values.  Will you clean or filling them? Explain. 
**Remember** to check the results of your code before passing to the next step

In [94]:
#I decided to fill them with 0 to translate that they have posts thouhg their count is lacking
import numpy as np
users_posts = users_posts.replace(to_replace = np.nan, value = 0) 
#other wat name_df.fillna(0) 

#### 10. Adjust the data types in order to avoid future issues. Which ones should be changed? 

In [97]:
posts_df["ViewCount"].dtypes

dtype('float64')

In [80]:
test=users.merge(posts, on = "userId")

# Refazer 9 e 10